# 读取原始excel

In [ ]:
try:
    df = pd.read_excel(path, skiprows=4)
    df = df[df.来电类型=='急救派车']
except:
    df = pd.read_excel(path)
    df = df[df.来电类型=='急救派车']
    
df.dropna(axis=0, subset = ["接车地址", "出车时间", "病史"], how='all', inplace=True)
df = df[~df.送达时间.isnull()]

# 时间

In [ ]:
df['受理时间'] = pd.to_datetime(df['受理时间'])
df['出车时间'] = pd.to_datetime(df['出车时间'])
df['到达现场时间'] = pd.to_datetime(df['到达现场时间'])
df['上车时间'] = pd.to_datetime(df['上车时间'])
df['送达时间'] = pd.to_datetime(df['送达时间'])

df['time1'] = (df['出车时间']-df['受理时间']).dt.seconds #受理调度时间
df['time2'] = (df['到达现场时间']-df['出车时间']).dt.seconds #去程在途时间
df['time3'] = (df['上车时间']-df['到达现场时间']).dt.seconds #现场停车时间
df['time4'] = (df['送达时间']-df['上车时间']).dt.seconds #返程在途时间
df['急救反应时间'] = df['time1']+df['time2']+df['time3']+df['time4']

# 与急救站地址表进行合并

In [ ]:
station = pd.read_csv('急救站_gps_WGSchecked.csv')
df['分站lng'] = -1
df['分站lat'] = -1
for s in df.所属分站.unique():
    lng, lat = station[station['120急救记录中的对应分站名称']==s].values[0][-2:]
    df.loc[df.所属分站==s,'分站lng'] = lng
    df.loc[df.所属分站==s,'分站lat'] = lat

# 年龄

In [ ]:
tmp = df[~df.病人年龄.isnull()]
tmp['年龄'] = tmp['病人年龄'].apply(lambda x:round(eval(x.replace('+','').replace('-','').replace(',','.'))))

In [ ]:
#统计各年龄数量
res = pd.DataFrame(tmp['年龄'].value_counts().sort_index().reset_index())
res.columns = ['年龄','数量']

In [ ]:
#统计各年龄段数量
ages = ['0-5', '5-18', '18-40', '40-60', '60-']

def age_to_str(age):
    if 0<= age < 5:
        return '00-05'
    if 5<= age < 18:
        return '05-18'
    if 18<= age < 40:
        return '18-40'
    if 40<= age < 60:
        return '40-60'
    if 60<= age:
        return '60-120'

res = pd.DataFrame(tmp['年龄'].apply(age_to_str).value_counts().sort_index().reset_index())
res.columns = ['年龄段','数量']

# 经纬度爬取

In [ ]:
x_pi = 3.14159265358979324 * 3000.0 / 180.0
pi = 3.1415926535897932384626  # π
a = 6378245.0  # 长半轴
ee = 0.00669342162296594323  # 扁率

def gcj02towgs84(lng, lat):
    """
    GCJ02(火星坐标系)转GPS84
    :param lng:火星坐标系的经度
    :param lat:火星坐标系纬度
    :return:
    """
    dlat = transformlat(lng - 105.0, lat - 35.0)
    dlng = transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * pi
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return [lng * 2 - mglng, lat * 2 - mglat]

def transformlat(lng, lat):
    ret = -100.0 + 2.0 * lng + 3.0 * lat + 0.2 * lat * lat + \
        0.1 * lng * lat + 0.2 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lat * pi) + 40.0 *
            math.sin(lat / 3.0 * pi)) * 2.0 / 3.0
    ret += (160.0 * math.sin(lat / 12.0 * pi) + 320 *
            math.sin(lat * pi / 30.0)) * 2.0 / 3.0
    return ret

def transformlng(lng, lat):
    ret = 300.0 + lng + 2.0 * lat + 0.1 * lng * lng + \
        0.1 * lng * lat + 0.1 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lng * pi) + 40.0 *
            math.sin(lng / 3.0 * pi)) * 2.0 / 3.0
    ret += (150.0 * math.sin(lng / 12.0 * pi) + 300.0 *
            math.sin(lng / 30.0 * pi)) * 2.0 / 3.0
    return ret


def get_addr(addr): 
    api_addr = 'http://api.map.baidu.com/geocoding/v3/?address='+addr+'&city=深圳&output=json&ret_coordtype=gcj02ll&ak=Wd1bXVfKlevK7MFQQ44olbAy4bFPUtxI'
    req = requests.get(api_addr)
    response = req.text
    response = json.loads(response)
    lng, lat = list(response['result']['location'].values())
    lng, lat = gcj02towgs84(lng, lat)
    return lng, lat


df['lng'] = 0
df['lat'] = 0

In [ ]:
for i in tqdm(range(len(df))):
    addr = df.loc[i,'接车地址']
    if df.loc[i,'lng'] == 0:
        try:
            lng, lat = get_addr(addr)        
            df.loc[i,'lng'] = lng
            df.loc[i,'lat'] = lat
        except:
            pass
#可能一天限额不够，跑完一轮需要第二天跑第二轮

In [ ]:
df = df[df.lng!=0] #用之前确保等于0的是爬不出地址的，而不是因为额度不够没爬到的

# 地址类别推断

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from transformers import BertModel, BertTokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class Model(nn.Module):
    def __init__(self, num_classes):
        super(Model, self).__init__()
        self.bert = BertModel.from_pretrained(MODEL_NAME, return_dict=False)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        _, out = self.bert(input_ids, attention_mask=attention_mask)
        y = self.fc(out)
        return y

In [ ]:
MODEL_NAME = 'hfl/chinese-roberta-wwm-ext-large'

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

model = Model(13).to(device)
model.load_state_dict(torch.load('roberta_bus_9513.pt', map_location=device), strict=False)
model.eval()

label = []

for d in tqdm(df.现场地址.values):
    text = d
    x = tokenizer(text, return_tensors="pt")
    input_ids = x['input_ids']
    attention_mask = x['attention_mask']
    input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
    output = model(input_ids, attention_mask).argmax(1).cpu().numpy()
    label.append(output)
    
df['地址类型'] = np.array(label).squeeze(-1)

labels = ['bus stop',
         'public',
         'work',
         'home',
         'transportation',
         'hotel',
         'clinic',
         'recreation',
         'jail',
         'nursing',
         'education',
         'street',
         'sport']
#也可以改成中文名

df['地址类型'] = df['地址类型'].apply(lambda x:labels[x])

# 疾病分类

In [ ]:
df['text'] = df['病人性别']+'[SEP]'+df['病人年龄']+'[SEP]'+\
            df['病情(主诉)']+'[SEP]'+df['病史']+'[SEP]'+df['主要体征']+'[SEP]'+\
            df['初步诊断']+'[SEP]'+df['补充诊断']
df['label1'] = -1
df['label2'] = -1

#老数据
train_df['text'] = train_df['病人性别']+'[SEP]'+train_df['病人年龄']+'[SEP]'+\
            train_df['病情(主诉)']+'[SEP]'+train_df['病史']+'[SEP]'+train_df['主要体征']+\
                '[SEP]'+train_df['初步诊断']+'[SEP]'+train_df['初步诊断2']

#新数据
df['text'] = df['性别']+'[SEP]'+df['age']+'[SEP]'+\
             df['主诉'].astype(str)+'[SEP]'+df['现病史'].astype(str)+'[SEP]'+\
             df['初步诊断'].astype(str)+'[SEP]'+df['补充诊断'].astype(str)+df['呼救原因']

In [ ]:
CFG = {
    'fold_num': 5,
    'seed': 42,
    'model': 'hfl/chinese-macbert-base',
    'max_len': 300, 
    'epochs': 10,
    'train_bs': 32, 
    'valid_bs': 32,
    'lr': 2e-5, 
    'num_workers': 4,
    'accum_iter': 1, 
    'weight_decay': 1e-6, 
    'device': 0,
}

In [ ]:
labels1 = ['儿科',
 '其他',
 '内分泌系统疾病',
 '创伤',
 '呼吸系统疾病',
 '妇产科',
 '心脏骤停',
 '心血管系统疾病',
 '感染性疾病',
 '泌尿系统疾病',
 '消化系统疾病',
 '理化中毒',
 '神经系统疾病',
 '精神病']
labels2 = ['儿科',
 '其他-休克',
 '其他-其他症状',
 '其他-意识不清',
 '其他-昏迷',
 '其他-死亡',
 '其他-胸闷',
 '内分泌系统疾病',
 '创伤-交通事故',
 '创伤-其他原因',
 '创伤-暴力事件',
 '创伤-跌倒',
 '创伤-高处坠落',
 '呼吸系统疾病',
 '妇产科',
 '心脏骤停',
 '心血管系统疾病-其他疾病',
 '心血管系统疾病-胸痛',
 '感染性疾病',
 '泌尿系统疾病',
 '消化系统疾病',
 '理化中毒',
 '神经系统疾病-其他疾病',
 '神经系统疾病-脑卒中',
 '精神病']

In [ ]:
class MyDataset(Dataset):
    def __init__(self, dataframe):
        self.df = dataframe
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        text = str(self.df.text.values[idx])
        
        label1 = int(self.df.label1.values[idx])
        label2 = int(self.df.label2.values[idx])

        coarse_label = label1
        fine_label = label2
 
        return text, coarse_label, fine_label

In [ ]:
class Model(nn.Module):

    def __init__(self, CFG):
        super(Model, self).__init__()
        self.bert = BertModel.from_pretrained(CFG['model'])
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(self.bert.config.hidden_size, 14)
        self.fc2 = nn.Linear(self.bert.config.hidden_size, 25)
 
    def forward(self, input_ids, attention_mask):
        text = self.bert(input_ids=input_ids, attention_mask=attention_mask)[1]
        y1 = self.fc1(text)
        y2 = self.fc2(text)
        return y1, y2

In [ ]:
def collate_fn(data):
    text = tokenizer([x[0] for x in data], padding='max_length', truncation=True, max_length=CFG['max_len'], return_tensors='pt')
    input_ids = text['input_ids']
    attention_mask = text['attention_mask']
    label1 = torch.LongTensor([x[1] for x in data])
    label2 = torch.LongTensor([x[2] for x in data])
    return input_ids, attention_mask, label1, label2

In [ ]:
test_set = MyDataset(df)
test_loader = DataLoader(test_set, batch_size=CFG['valid_bs'], collate_fn=collate_fn, shuffle=False, num_workers=4)

tokenizer = BertTokenizer.from_pretrained(CFG['model'])

In [ ]:
model = Model(CFG).to(device)
model.load_state_dict(torch.load('../疾病分类/{}.pt'.format(CFG['model'].split('/')[-1])))
model.eval()
    
    
pred = []
with torch.no_grad():
    tk = tqdm(test_loader, total=len(test_loader), position=0, leave=True)
    for step, batch in enumerate(tk):
        input_ids, attention_mask, y1, y2 = [x.to(device) for x in batch]

        _,output = model(input_ids, attention_mask)
        output = output.softmax(1).cpu().numpy()
        
        pred.extend(output)

In [ ]:
pred = np.array(pred)
df['disease_label'] = pred.argmax(1)
df['disease_label'] = df['disease_label'].apply(lambda x:labels2[x])

# 一些基本属性的统计方法

性别 value_counts()

疾病类型 value_counts().sort_values()

时、日、月 pd.DatetimeIndex(df['受理时间']).hour/weekday/month.value_counts().sort_index()

地址坐标 跳过，应该在地图可视化里

地址类型 value_counts().sort_values()

急救反应时间等 hist直方图

# 关联分析

In [ ]:
# 时间-空间
for a in df.addr_label.unique():
    tmp = df[df.addr_label==a]
    res = pd.DataFrame(pd.DatetimeIndex(tmp['受理时间']).hour.value_counts().sort_index().reset_index())
    res.columns = ['小时','数量']
    #hour可改成weekday,month

# 时间-性别
for a in df.病人性别.unique(): #for a in ['男','女']:
    tmp = df[df.病人性别==a]
    res = pd.DataFrame(pd.DatetimeIndex(tmp['受理时间']).hour.value_counts().sort_index().reset_index())
    res.columns = ['小时','数量']
    #hour可改成weekday,month

# 时间-性别
df['年龄段'] = df.年龄.apply(age_to_str)
for a in df.年龄段.unique():
    tmp = df[df.年龄段==a]
    res = pd.DataFrame(pd.DatetimeIndex(tmp['受理时间']).hour.value_counts().sort_index().reset_index())
    res.columns = ['小时','数量']
    #hour可改成weekday,month

# 时间-病理学
for a in df.disease_label.unique():
    tmp = df[df.disease_label==a]
    res = pd.DataFrame(pd.DatetimeIndex(tmp['受理时间']).hour.value_counts().sort_index().reset_index())
    res.columns = ['小时','数量']
    #hour可改成weekday,month

# 时间-效率
df['hour'] = pd.DatetimeIndex(df['受理时间']).hour
res = []
for x in df.hour.unique():
    y = df[df.hour==x]['time'].mean()
    res.append([x,y])
res = pd.DataFrame(res)
res.columns = ['hour','急救反应时间']
#hour可改成weekday,month

# 时间-急救站
for a in df.所属分站.unique():
    tmp = df[df.所属分站==a]
    res = pd.DataFrame(pd.DatetimeIndex(tmp['受理时间']).hour.value_counts().sort_index().reset_index())
    res.columns = ['小时','数量']
    #hour可改成weekday,month

## 基本思路

如果不涉及效率，那么就是两个属性x和y。对每个取值x遍历，计算x下y的value_counts()

如果涉及效率，把效率（各环节时间）当做y。对每个取值x遍历，计算x下y平均值